# Module 2 | Homework Solution

## Q1. Embedding the query
Embed the query: 'I just discovered the course. Can I join now?'. Use the 'jinaai/jina-embeddings-v2-small-en' model.

You should get a numpy array of size 512.

What's the minimal value in this array?

In [21]:
from fastembed import TextEmbedding
import numpy as np

In [3]:
query = 'I just discovered the course. Can I join now?'
model = TextEmbedding(model_name='jinaai/jina-embeddings-v2-small-en')
embedding_query = list(model.embed(query))[0]

In [4]:
embedding_query.shape

(512,)

In [5]:
#What's the minimal value in this array?
embedding_query.min()

np.float64(-0.11726373885183883)

## Q2. Cosine similarity with another vector

Now let's embed this document:
```python
doc = 'Can I still join the course after the start date?'
```

What's the cosine similarity between the vector for the query and the vector for the document?

In [6]:
doc = 'Can I still join the course after the start date?'

In [9]:
doc_embedding = list(model.embed(doc))[0]
doc_embedding.shape

(512,)

In [10]:
embedding_query.dot(doc_embedding)

np.float64(0.9008528895674548)

### Answer 2: 0.90

## Q3. Ranking by cosine similarity

Compute the embeddings for the text field, and compute the cosine between the query vector and all the documents.

What's the document index with the highest similarity? (Indexing starts from 0)

In [11]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [22]:
documents_embeddings = np.array([list(model.embed(doc['text']))[0] for doc in documents])

In [24]:
embedding_query.dot(documents_embeddings.T).argmax()

np.int64(1)

### Answer 3: 1

## Q4. Ranking by cosine, version two
Now let's calculate a new field, which is a concatenation of question and text:

```python
full_text = doc['question'] + ' ' + doc['text']
```
Embed this field and compute the cosine between it and the query vector. What's the highest scoring document?

In [ ]:
doc_embeddings = np.array([list(model.embed(doc['question'] + ' ' + doc['text']))[0] for doc in documents])

In [26]:

embedding_query.dot(doc_embeddings.T).argmax()

np.int64(0)

is different from 3 because the question and the text are now part of the same vector.

### Answer 4: 0

## Q5. Selecting the embedding model
Now let's select a smaller embedding model. What's the smallest dimensionality for models in fastembed?

In [30]:
import json

In [42]:
models = TextEmbedding.list_supported_models()
models.sort(key=lambda x: x['dim'])
print(models[0]['dim'])

384


### Answer 5: 384

## Q6. Indexing with qdrant (2 points)
For the last question, we will use more documents.

We will select only FAQ records from our ml zoomcamp:

```python
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
```

Add them to qdrant using the model form Q5.

When adding the data, use both question and answer fields:

```python
text = doc['question'] + ' ' + doc['text']
```
After the data is inserted, use the question from Q1 for querying the collection.

What's the highest score in the results? (The score for the first returned record):



In [68]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

documents[93]

{'text': "Should we perform EDA on the base of train or train+validation or train+validation+test dataset?\nIt's indeed good practice to only rely on the train dataset for EDA. Including validation might be okay. But we aren't supposed to touch the test dataset, even just looking at it isn't a good idea. We indeed pretend that this is the future unseen data\nAlena Kniazeva",
 'section': '3. Machine Learning for Classification',
 'question': 'What data should be used for EDA?',
 'course': 'machine-learning-zoomcamp'}

In [47]:
from qdrant_client import QdrantClient, models

# Initialize the client
client = QdrantClient(":memory:")

In [48]:
client.set_model("BAAI/bge-small-en")

Fetching 5 files: 100%|██████████| 5/5 [03:49<00:00, 45.90s/it] 


In [73]:
collection_name = "mlzoomcamp-rag"

client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=384,
        distance=models.Distance.COSINE
    )
)

True

In [69]:
points = []
id = 0

for doc in documents:
  point = models.PointStruct(
    id=id,
    vector=models.Document(text=doc['question'] + ' ' + doc['text'], model="BAAI/bge-small-en"),
    payload={
      "text": doc['text'],
      "section": doc['section'],
      "course": doc['course'],
      "question": doc['question']
    } #save all needed metadata fields
  )
  points.append(point)

  id += 1

In [74]:
client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [75]:
def search(query, limit=1):
  results = client.query_points(
    collection_name=collection_name,
    query=models.Document(
      text=query,
      model="BAAI/bge-small-en"
    ),
    limit=limit,
    with_payload=True
  )
  return results

In [76]:
query = 'I just discovered the course. Can I join now?'

In [92]:
result = search(query).model_dump_json()
json.loads(result)

{'points': [{'id': 14,
   'version': 0,
   'score': 0.8703173344744922,
   'payload': {'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
    'section': 'General course-related questions',
    'course': 'machine-learning-zoomcamp',
    'question': 'The course has already started. Can I still join it?'},
   'vector': None,
   'shard_key': None,
   'order_value': None}]}

### Answer 6: 0.87